Sentiment Analysis

# **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load Dataset**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_s.csv')

In [ ]:
df

,sentimen,tweet,jumlah_kata,frekuensi_tanda_baca,angka,emails_found,sentimen_mapped,tweet_token
0,negatif,prabowo indonesia dihargai bangsa asing berita...,19,"Counter({'/': 5, '@': 2, '?': 2, '!': 1, ',': ...","['1117575436337160192', '21']",[],0,"['prabowo', 'indonesia', 'harga', 'bangsa', 'a..."
1,netral,batuan langka tasbih jokowi hadiah habib luthf...,11,"Counter({'/': 3, ',': 1, '?': 1, ':': 1, '.': 1})",['2'],[],1,"['batu', 'langka', 'tasbih', 'jokowi', 'hadiah..."
2,netral,era jokowi ekonomi indonesia twitter com w,11,"Counter({'.': 3, '#': 3, ',': 1, '/': 1})","['01', '2']",[],1,"['era', 'jokowi', 'ekonomi', 'indonesia', 'twi..."
3,positif,sumatera asian games berdampak pd ekonomi dipr...,17,"Counter({',': 3, '.': 1, '!': 1})","['18', '5']",[],2,"['sumatera', 'asi', 'games', 'dampak', 'pd', '..."
4,negatif,negara ngutang bngun infrastruktur dipake masy...,34,"Counter({',': 3, '.': 1})",[],[],0,"['negara', 'ngutang', 'bngun', 'infrastruktur'..."
...,...,...,...,...,...,...,...,...
1810,netral,negarawan sejati sll bangga mengedepankan harg...,12,Counter({'#': 1}),['2019'],[],1,"['negarawan', 'sejati', 'sll', 'bangga', 'depa..."
1811,netral,ceramah damai indonesiaku perekonomian membaik,11,"Counter({'.': 3, '#': 1})","['1', '2']",[],1,"['ceramah', 'damai', 'indonesia', 'ekonomi', '..."
1812,netral,bangun bangsa mendukung perekonomian negara jo...,14,"Counter({'/': 5, '.': 2, '!': 1, ':': 1})",['1111643312241295363'],[],1,"['bangun', 'bangsa', 'dukung', 'ekonomi', 'neg..."
1813,netral,bantu majukan perekonomian bangsa jokowi yuk h...,10,"Counter({'/': 5, ',': 1, '!': 1, ':': 1, '.': 1})",['1113655944955588610'],[],1,"['bantu', 'maju', 'ekonomi', 'bangsa', 'jokowi..."


In [ ]:
df=df[['sentimen','tweet_token']]

# **Modeling**

In [ ]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


# Fungsi untuk menghapus URL
def remove_urls(tokens):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return [url_pattern.sub('', token) for token in tokens if not url_pattern.match(token)]

# Fungsi untuk menghapus angka
def remove_numbers(tokens):
    number_pattern = re.compile(r'\b\d+\b')
    return [token for token in tokens if not number_pattern.match(token)]

# Preprocessing
df['tweet_token'] = df['tweet_token'].apply(remove_urls).apply(remove_numbers)

# Mengonversi label sentimen ke bentuk numerik
label_encoder = LabelEncoder()
df['sentimen'] = label_encoder.fit_transform(df['sentimen'])

# Menggabungkan token menjadi satu string untuk tokenisasi
df['tweet_token_str'] = df['tweet_token'].apply(lambda x: ' '.join(x))

# Tokenisasi
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tweet_token_str'])
X = tokenizer.texts_to_sequences(df['tweet_token_str'])

# Padding sequences
X = pad_sequences(X, padding='post')

# Label
y = to_categorical(df['sentimen'])

# Split data menjadi data latih dan data uji
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membangun model LSTM
# Definisi model LSTM
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # Karena ada 3 kategori sentimen

# Kompilasi model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


<ipython-input-8-11830fb77496>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_token'] = df['tweet_token'].apply(remove_urls).apply(remove_numbers)
<ipython-input-8-11830fb77496>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentimen'] = label_encoder.fit_transform(df['sentimen'])
<ipython-input-8-11830fb77496>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

Epoch 1/10
46/46 [==============================] - 51s 996ms/step - loss: 1.1014 - accuracy: 0.3058 - val_loss: 1.1010 - val_accuracy: 0.3223
Epoch 2/10
46/46 [==============================] - 43s 943ms/step - loss: 1.1024 - accuracy: 0.3182 - val_loss: 1.1037 - val_accuracy: 0.3251
Epoch 3/10
46/46 [==============================] - 39s 829ms/step - loss: 1.1006 - accuracy: 0.3189 - val_loss: 1.0985 - val_accuracy: 0.3251
Epoch 4/10
46/46 [==============================] - 53s 1s/step - loss: 1.0995 - accuracy: 0.3375 - val_loss: 1.1001 - val_accuracy: 0.3223
Epoch 5/10
11/46 [======>.......................] - ETA: 1:09 - loss: 1.1003 - accuracy: 0.3267

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

NameError: name 'y_test_classes' is not defined

In [ ]:
from sklearn.metrics import precision_score
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
from sklearn.metrics import recall_score
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
test_data = df.iloc[np.arange(len(y_test))]

In [ ]:
test_data

In [ ]:
test_data.to_csv('test_data.csv', index=False)

In [ ]:
from google.colab import files
files.download('test_data.csv')

# **Hyperparameter Tuning**

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from keras_tuner import RandomSearch

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                        output_dim=hp.Int('embedding_dim', min_value=50, max_value=200, step=50),
                        input_length=X.shape[1]))
    model.add(SpatialDropout1D(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=50, max_value=200, step=50),
                   dropout=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1),
                   recurrent_dropout=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Inisialisasi Keras Tuner dengan RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='sentiment_analysis_tuning'
)

# Melakukan pencarian hyperparameter
tuner.search(X, y, epochs=5, validation_split=0.2)

# Menampilkan hasil terbaik
best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()

Trial 5 Complete [00h 15m 32s]
val_accuracy: 0.3489439884821574

Best val_accuracy So Far: 0.3581267297267914
Total elapsed time: 00h 43m 30s
Results summary
Results in my_dir/sentiment_analysis_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
embedding_dim: 200
dropout_rate: 0.2
lstm_units: 100
Score: 0.3581267297267914

Trial 0 summary
Hyperparameters:
embedding_dim: 200
dropout_rate: 0.2
lstm_units: 50
Score: 0.3489439884821574

Trial 1 summary
Hyperparameters:
embedding_dim: 50
dropout_rate: 0.30000000000000004
lstm_units: 50
Score: 0.3489439884821574

Trial 4 summary
Hyperparameters:
embedding_dim: 200
dropout_rate: 0.4
lstm_units: 200
Score: 0.3489439884821574

Trial 2 summary
Hyperparameters:
embedding_dim: 50
dropout_rate: 0.2
lstm_units: 150
Score: 0.3397612472375234
